# 4. Estimating the Brock and Mirman (1972) model by GMM

In [134]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [245]:
mcseries = pd.read_csv("MacroSeries.txt", header = None,squeeze=True, names = ["c","k","w","r"])

c = mcseries["c"].as_matrix()
k = mcseries["k"].as_matrix()
w = mcseries["w"].as_matrix()
r = mcseries["r"].as_matrix()

In [246]:
k

array([  8040697.        ,   7595071.76117567,   8323735.10441439,
         6613792.41053042,   5632135.94271214,   5794291.99147403,
         5889252.24203382,   6139431.32809391,   6414807.80778456,
         6255021.24408637,   6918829.21668457,   7153046.32196178,
         6920333.9840067 ,   7813609.15822999,   9540329.63179025,
         8292049.71479493,   7603585.3609027 ,   7800005.42815411,
         7221336.36934274,   5857840.07545773,   6488639.94941073,
         6353698.86290216,   5815159.47259902,   5205224.6587504 ,
         5531034.09979365,   5886818.89478422,   5545938.72815041,
         5123678.33190049,   5841991.56255407,   6337258.61883496,
         6247566.31186923,   6576043.29440867,   7387253.79111871,
         8184884.57228105,   8054508.55912559,  10636793.05938092,
        12836302.56042412,  12623369.1992458 ,  13212559.75196111,
        12771028.14723333,   9635997.61034672,   7717914.40313058,
         6637588.58291581,   6594864.87683528,   6105903.33673

SyntaxError: invalid syntax (<ipython-input-199-f87d602e4b73>, line 1)

### (a)

This example is neat because we use the FOCs and zero-expectations as moments. In the model they are supposed to be zero, which means all we are left with is calculating the data moments. Let's start.

In [200]:
mcseries.mean()

c    1.052085e+07
k    7.472545e+06
w    1.044521e+07
r    1.016947e+00
dtype: float64

Since all equations include two time periods, we have one observation less for the equations than for variables.

In [201]:
# Recall from Problem 2:
def zvia4(r, k, alpha):
    return np.log(r/((alpha) * (k**(alpha - 1))))

In [247]:
def data_moments( c, k, w, r, alpha, beta, rho, mu):
    z = zvia4(r, k, alpha)
    c = mcseries["c"].as_matrix()
    k = mcseries["k"].as_matrix()
    w = mcseries["w"].as_matrix()
    r = mcseries["r"].as_matrix()
    
    eq6 = z[1:] - rho * z[:-1] - (1 - rho) * mu
    eq7 = (z[1:] - rho * z[:-1] - (1 - rho) * mu) * z[:-1]
    #print(type(k),type(c))
    eq8 = (beta * alpha * np.exp(z[1:]) *\
           (k[1:]**(alpha - 1))*c[:-1]/c[1:]) - 1
    eq9 = ((beta * alpha * np.exp(z[1:]) *\
            (k[1:]**(alpha - 1))*c[:-1]/c[1:]) - 1) * w[:-1]
    
    moments = np.array(([eq6.mean(), eq7.mean(), eq8.mean(), eq9.mean()]))
    
    return moments

'''def err_vec( c, k, w, r, alpha, beta, rho, mu, simple):
    moments = data_moments( c, k, w, r, alpha, beta, rho, mu)
    moms_model = np.zeros_like(moms_data)
    if simple:
        err_vec = moms_model - moms_data
    else:
        err_vec = (moms_model - moms_data) / moms_data
    return err_vec'''

def crit(params, *args):
    alpha, beta, rho, mu = params
    c, k, w, r = args
    W = np.eye(4)
    err = data_moments( c, k, w, r, alpha, beta, rho, mu)
    crit_val = np.dot(np.dot(err.T, W), err) 
    return crit_val 

In [256]:
alpha_init = 0.25
beta_init = 0.99
rho_init = -0.001
mu_init = 1


params_init = np.array([alpha_init, beta_init, rho_init, mu_init])
args = (c, k, w, r)

bounds = ((0,1),(0,1),(-1,1),(0,None))
results = opt.minimize(crit, params_init, args=args,
                       method ='L-BFGS-B', bounds=bounds)
alpha_GMM, beta_GMM, rho_GMM, mu_GMM = results.x
print(results)
print('alpha_GMM = ', alpha_GMM, 'beta_GMM = ', beta_GMM, 'rho_GMM = ', rho_GMM, 'mu_GMM = ', mu_GMM)
print('Minimized Criterion:', crit(results.x, *args))

      fun: 0.0027698241119015012
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([  3.27877274e-03,   3.52898743e+00,  -6.60682446e-06,
         2.49657066e-06])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 80
      nit: 7
   status: 0
  success: True
        x: array([  2.52135736e-01,   9.89999995e-01,   1.82671569e-03,
         1.32107590e+01])
alpha_GMM =  0.252135736431 beta_GMM =  0.989999995 rho_GMM =  0.00182671568732 mu_GMM =  13.2107589559
Minimized Criterion: 0.0027698241119


'from scipy import linalg as la\ndist = 1\nalpha_init = 0.25\nbeta_init = 0.99\nrho_init = 0.01\nmu_init = 1\nk = 1\nargs = (c, k, w, r)\nbounds = ((0,1),(0,1),(-1,1),(0,None))\nwhile dist > 1e-2 and k <10:\n    c = mcseries["c"].as_matrix()\n    k = mcseries["k"].as_matrix()\n    w = mcseries["w"].as_matrix()\n    r = mcseries["r"].as_matrix()\n    params_init = np.array([alpha_init, beta_init, rho_init, mu_init])\n    \n    results = opt.minimize(crit, params_init, args=args,\n                       method =\'L-BFGS-B\', bounds=bounds)\n    alpha_GMM, beta_GMM, rho_GMM, mu_GMM = results.x\n    print(\'alpha_GMM = \', alpha_GMM, \'beta_GMM = \', beta_GMM, \'rho_GMM = \', rho_GMM, \'mu_GMM = \', mu_GMM)\n    dist = la.norm(la.norm(np.array(([alpha_GMM - alpha_init,\n                              beta_GMM - beta_init,\n                              rho_GMM - rho_init,\n                              mu_GMM - mu_init])) ))\n    alpha_init, beta_init, rho_init, mu_init = alpha_GMM, beta_GM

I iterated a couple of times but the result is not great:

In [242]:
alpha_init, beta_init, rho_init, mu_init = alpha_GMM, beta_GMM, rho_GMM, mu_GMM


params_init = np.array([alpha_init, beta_init, rho_init, mu_init])
args = (c, k, w, r)

bounds = ((0,1),(0,1),(-1,1),(0,None))
results = opt.minimize(crit, params_init, args=args,
                       method ='L-BFGS-B', bounds=bounds)
alpha_GMM, beta_GMM, rho_GMM, mu_GMM = results.x
print(results)
print('alpha_GMM = ', alpha_GMM, 'beta_GMM = ', beta_GMM, 'rho_GMM = ', rho_GMM, 'mu_GMM = ', mu_GMM)
print('Minimized Criterion:', crit(results.x, *args))

      fun: 19.511085536145725
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -4.35656283e+01,   1.53396769e+04,  -4.33372577e+01,
         8.31414368e+00])
  message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 105
      nit: 0
   status: 2
  success: False
        x: array([ 0.98249999,  0.99      ,  0.1       ,  5.        ])
alpha_GMM =  0.982499991428 beta_GMM =  0.989999995069 rho_GMM =  0.1 mu_GMM =  5.0
Minimized Criterion: 19.5110855361


In [ ]:
It seems to have gotten itself into a failure to converge.

In [267]:
alpha_init = 0.25
beta_init = 0.99
rho_init = -0.001
mu_init = 1


params_init = np.array([alpha_init, beta_init, rho_init, mu_init])
args = (c, k, w, r)

bounds = ((0,1),(0,1),(-1,1),(0,None))
results = opt.minimize(crit, params_init, args=args,
                       method ='L-BFGS-B', bounds=bounds)
alpha_GMM, beta_GMM, rho_GMM, mu_GMM = results.x
print(results)
print('alpha_GMM = ', alpha_GMM, 'beta_GMM = ', beta_GMM, 'rho_GMM = ', rho_GMM, 'mu_GMM = ', mu_GMM)
print('Minimized Criterion:', crit(results.x, *args))


      fun: 0.0027698253881187789
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -3.71167758e-03,   3.33089698e+00,  -6.59997230e-06,
        -1.78312226e-06])
  message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 270
      nit: 6
   status: 2
  success: False
        x: array([  2.52135736e-01,   9.89999995e-01,   1.82671607e-03,
         1.32107598e+01])
alpha_GMM =  0.252135736165 beta_GMM =  0.989999995 rho_GMM =  0.00182671606624 mu_GMM =  13.2107597818
Minimized Criterion: 0.00276982513512


In [268]:
c = mcseries["c"].as_matrix()
k = mcseries["k"].as_matrix()
w = mcseries["w"].as_matrix()
r = mcseries["r"].as_matrix()
params_init = np.array([alpha_init, beta_init, rho_init, mu_init])
    
results = opt.minimize(crit, params_init, args=args,
                       method ='L-BFGS-B', bounds=bounds)
alpha_GMM, beta_GMM, rho_GMM, mu_GMM = results.x
print('alpha_GMM = ', alpha_GMM, 'beta_GMM = ', beta_GMM, 'rho_GMM = ', rho_GMM, 'mu_GMM = ', mu_GMM)
dist = la.norm(la.norm(np.array(([alpha_GMM - alpha_init,
                                  beta_GMM - beta_init,
                                  rho_GMM - rho_init,
                                  mu_GMM - mu_init])) ))
alpha_init, beta_init, rho_init, mu_init = alpha_GMM, beta_GMM, rho_GMM, mu_GMM


alpha_GMM =  0.252135736165 beta_GMM =  0.989999995 rho_GMM =  0.00182671606624 mu_GMM =  13.2107597818


In [269]:
c = mcseries["c"].as_matrix()
k = mcseries["k"].as_matrix()
w = mcseries["w"].as_matrix()
r = mcseries["r"].as_matrix()
params_init = np.array([alpha_init, beta_init, rho_init, mu_init])
    
results = opt.minimize(crit, params_init, args=args,
                       method ='L-BFGS-B', bounds=bounds)
alpha_GMM, beta_GMM, rho_GMM, mu_GMM = results.x
print('alpha_GMM = ', alpha_GMM, 'beta_GMM = ', beta_GMM, 'rho_GMM = ', rho_GMM, 'mu_GMM = ', mu_GMM)
dist = la.norm(la.norm(np.array(([alpha_GMM - alpha_init,
                                  beta_GMM - beta_init,
                                  rho_GMM - rho_init,
                                  mu_GMM - mu_init])) ))
alpha_init, beta_init, rho_init, mu_init = alpha_GMM, beta_GMM, rho_GMM, mu_GMM



alpha_GMM =  0.252135736165 beta_GMM =  0.989999995 rho_GMM =  0.00182671606624 mu_GMM =  13.2107597818


In [270]:
c = mcseries["c"].as_matrix()
k = mcseries["k"].as_matrix()
w = mcseries["w"].as_matrix()
r = mcseries["r"].as_matrix()
params_init = np.array([alpha_init, beta_init, rho_init, mu_init])
    
results = opt.minimize(crit, params_init, args=args,
                       method ='L-BFGS-B', bounds=bounds)
alpha_GMM, beta_GMM, rho_GMM, mu_GMM = results.x
print('alpha_GMM = ', alpha_GMM, 'beta_GMM = ', beta_GMM, 'rho_GMM = ', rho_GMM, 'mu_GMM = ', mu_GMM)
dist = la.norm(la.norm(np.array(([alpha_GMM - alpha_init,
                                  beta_GMM - beta_init,
                                  rho_GMM - rho_init,
                                  mu_GMM - mu_init])) ))
alpha_init, beta_init, rho_init, mu_init = alpha_GMM, beta_GMM, rho_GMM, mu_GMM



alpha_GMM =  0.252135736165 beta_GMM =  0.989999995 rho_GMM =  0.00182671606624 mu_GMM =  13.2107597818


In [271]:
results

      fun: 0.0027698253881187789
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -3.71167758e-03,   3.33089698e+00,  -6.59997230e-06,
        -1.78312226e-06])
  message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 105
      nit: 0
   status: 2
  success: False
        x: array([  2.52135736e-01,   9.89999995e-01,   1.82671607e-03,
         1.32107598e+01])

This result seems to be stable as well, but it is still close to the initial guess, so perhaps the estimation just fell into a well